<a href="https://colab.research.google.com/github/JoshFCooper415/JoshFCooper415/blob/main/DenseNetIplimentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dense
from tensorflow.keras.layers import AveragePooling2D, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.layers import ReLU, concatenate
import tensorflow.keras.backend as K
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

In [4]:
data = tf.keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = data.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
# input  
input = Input(shape =(32,32, 3))
filters = 32

169017344/169001437 [==============================] - 2s 0us/step


In [5]:
def batchNorm_relu (x, filters, kernel=1, strides=1):
  x = BatchNormalization()(x)
  x = ReLU()(x)
  x = Conv2D (filters, kernel, strides=strides, padding ='same')(x)
  x = Dropout(.2)(x)
  return x
def dense_block (x, repetition):

  for _ in range (repetition):
    y = batchNorm_relu(x, 4*filters)
    y = batchNorm_relu(y, filters, 3)
    x = concatenate([y,x])
  return x
def transition_layer (x):
  batchNorm_relu(x, K.int_shape(x)[-1]//2)
  x = AveragePooling2D(pool_size=(2, 2), strides = 2, padding ='same')(x)
  return x

In [6]:
with tf.device('/device:GPU:0'):
      x = Conv2D (filters =64, kernel_size =7, padding ='same', activation='relu')(input)
      x = MaxPool2D (3, strides =2, padding ='same')(x)

      for repetition in [6,12,24,16]:     
              d = dense_block(x, repetition)
              x = transition_layer(d)
      x = GlobalAveragePooling2D()(d)
      x = Dropout(.5)(x) #testing a dropout here

      output = Dense(units = 100, activation ='softmax')(x)
      
      model = Model (inputs=input, outputs =output)
      model.summary()
      loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
      loss_fn(y_train[:1], output)
      model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 64)   9472        ['input_1[0][0]']                
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 16, 16, 64)   0           ['conv2d[0][0]']                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 16, 16, 64)  256         ['max_pooling2d[0][0]']          
 alization)                                                                                   

In [15]:
model.fit(x_train, y_train, epochs=75, batch_size=500, validation_split=0.1)
model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/75
90/90 [==============================] - 86s 643ms/step - loss: 3.9788 - accuracy: 0.1076 - val_loss: 5.1320 - val_accuracy: 0.0206
Epoch 2/75
90/90 [==============================] - 54s 604ms/step - loss: 3.2591 - accuracy: 0.2095 - val_loss: 5.0193 - val_accuracy: 0.0536
Epoch 3/75
90/90 [==============================] - 54s 604ms/step - loss: 2.8678 - accuracy: 0.2787 - val_loss: 4.2526 - val_accuracy: 0.1078
Epoch 4/75
90/90 [==============================] - 56s 618ms/step - loss: 2.5901 - accuracy: 0.3336 - val_loss: 4.1831 - val_accuracy: 0.1626
Epoch 5/75
90/90 [==============================] - 54s 604ms/step - loss: 2.3902 - accuracy: 0.3733 - val_loss: 6.4400 - val_accuracy: 0.1048
Epoch 6/75
90/90 [==============================] - 54s 604ms/step - loss: 2.2075 - accuracy: 0.4132 - val_loss: 3.2016 - val_accuracy: 0.3012
Epoch 7/75
90/90 [==============================] - 54s 604ms/step - loss: 2.0507 - accuracy: 0.4470 - val_loss: 3.8256 - val_accuracy: 0.2604

[4.513051509857178, 0.511900007724762]